In [1]:
import tensorflow as tf
import pandas as  pd
import numpy as np
import random
import io
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)
tf.config.experimental.set_memory_growth(gpu[1], True)

In [2]:
df = pd.read_csv("df_train_clean.csv")
df_disaster = df.text_clean[df.target == 1].reset_index(drop=True)
text = []
for row in range(len(df_disaster)):
    text.append(df_disaster[row])
text = ",".join(text).replace(","," \n ")
print(text[:500])

our deeds are the reason of this hastag earthquake may allah forgive us all 
 forest fire near la ronge sask canada 
 all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected 
 13000 people receive hastag wildfires evacuation orders in california 
 just got sent this photo from ruby hastag alaska as smoke from hastag wildfires pours into a school 
 hastag rockyfire update  california hwy 20 closed in both directions due to


In [3]:
vocab = sorted(set(text))
char_to_ind = {char:i for i, char in enumerate(vocab)}
ind_to_char = np.array(vocab)
encoded_text = np.array([char_to_ind[c] for c in text])

In [4]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(10):
     print(ind_to_char[i.numpy()])


o
u
r
 
d
e
e
d
s
 


In [5]:
seq_len = len(vocab)
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [6]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt
    
dataset = sequences.map(create_seq_targets)

for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[26 32 29  1 15 16 16 15 30  1 12 29 16  1 31 19 16  1 29 16 12 30 26 25
  1 26 17  1 31 19 20 30  1 19 12 30 31 12 18  1 16 12 29 31 19 28 32 12
 22 16  1]
our deeds are the reason of this hastag earthquake 


[32 29  1 15 16 16 15 30  1 12 29 16  1 31 19 16  1 29 16 12 30 26 25  1
 26 17  1 31 19 20 30  1 19 12 30 31 12 18  1 16 12 29 31 19 28 32 12 22
 16  1 24]
ur deeds are the reason of this 

In [7]:
# Batch size
batch_size = 50

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)


In [8]:

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(tf.keras.layers.LSTM(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dropout(0.2))
    # Final Dense Layer to Predict
    model.add(tf.keras.layers.Dense(vocab_size))
    model.compile(optimizer='adam', loss=loss) 
    return model

# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 64
# Number of RNN units
rnn_neurons =4154

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

#Create the model
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)


#Train the model
epochs = 60
model.fit(dataset,epochs=epochs)

Epoch 1/60
101/101 [==============================] - 20s 165ms/step - loss: 3.7836
Epoch 2/60
101/101 [==============================] - 17s 165ms/step - loss: 2.7340
Epoch 3/60
101/101 [==============================] - 17s 164ms/step - loss: 2.5016
Epoch 4/60
101/101 [==============================] - 17s 164ms/step - loss: 2.3525
Epoch 5/60
101/101 [==============================] - 17s 165ms/step - loss: 2.2290
Epoch 6/60
101/101 [==============================] - 17s 164ms/step - loss: 2.1115
Epoch 7/60
101/101 [==============================] - 17s 164ms/step - loss: 1.9796
Epoch 8/60
101/101 [==============================] - 17s 165ms/step - loss: 1.8482
Epoch 9/60
101/101 [==============================] - 17s 167ms/step - loss: 1.7279
Epoch 10/60
101/101 [==============================] - 17s 165ms/step - loss: 1.6222
Epoch 11/60
101/101 [==============================] - 17s 165ms/step - loss: 1.5168
Epoch 12/60
101/101 [==============================] - 17s 167ms/step - lo

In [19]:
from tensorflow.keras.models import load_model
import random
model.save('tweets_gen.h5') 

#Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights.
#Then call .build() on the mode

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('tweets_gen.h5')
model.build(tf.TensorShape([1, None]))


def generate_text(model, start_seed,gen_size=100,temp=0.95):
    '''
    model: Trained Model to Generate Text
    start_seed: Intial Seed text in string form
    gen_size: Number of characters to generate
    Basic idea behind this function is to take in some seed text, format it so
    that it is in the correct shape for our network, then loop the sequence as
    we keep adding our own predicted characters. Similar to our work in the RNN
    time series problems.
    '''
    # Number of characters to generate
    num_generate = gen_size
    # Vecotrizing starting seed text
    input_eval = [char_to_ind[s] for s in start_seed]
    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)
    # Empty list to hold resulting generated text
    text_generated = []
    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Higher probability == lesss surprising/ more expected
    # Lower temperature == more surprising / less expected
    temperature = temp
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
      # Generate Predictions
      predictions = model(input_eval)
      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)
      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)
      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])
    return (start_seed + ''.join(text_generated))


print(generate_text(model,random.choice(text.split()),gen_size=5000).replace('hastag','#'))

# news # chech # bitcoing # blockchain the midsouth and midwest   # weather 
 rock springs ûò residents land cry spreading fire # nightbeat veronicadly outbreak 
 10th death confirmed in legionnaires outbreak in southbbombed port north robat 
 bin laden family plane crashed after avoiding microlight and landing too far down runway 
 this guy bought mestrump is a climate denier algae bloom in the pacific liber now 
 bluebirddenver # fettilootch i # yyc # yyc # abstorm 
 ready for backern california   g 
 there was a small earthquake in la but dont worry emmy rossum is fine 
 sure the cameroon repatriating about 12000 police post in udhampur 2 police look back police post in udhampur 2 spos injured  livemintag zo # virwill and mainline train derails 
 hotel evacuated after landslide in the italian alps   via todayngr 
 faan giver in a layer of grit 
 sandstorm woo hoo 
 socialwots militants attack police post 2 spos injuredû   # just is too short 
 in this fragile global economy consitne